In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow import keras
%pip install git+https://github.com/keras-team/keras-preprocessing.git
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import gc

In [ ]:
import pandas as pd

img_dir = '/content/drive/My Drive/CelebA/Img/img_align_celeba_32500'
# Load labels.csv
labels_df = pd.read_csv('/content/drive/My Drive/CelebA/Anno/labels_even_clean.csv', header=0, names=['Filename', 'High_Cheekbones', 'Mouth_Slightly_Open', 'Smiling'])

In [ ]:
train, valid = train_test_split(labels_df, test_size = 0.3)
valid, test = train_test_split(valid, test_size=0.2)
train_features = train[["High_Cheekbones", "Mouth_Slightly_Open","Smiling"]]
# print length of train, valid and test
print("train: ", len(train))
print("valid: ", len(valid))
print("test: ", len(test))
#train,valid,test

In [ ]:
# remove a random row from train
train = train.drop(train.sample().index)

In [ ]:
'''import pandas as pd
import re

df = labels_df

# Function to check if a filename is valid
def is_valid_filename(filename):
    pattern = r'^\d{6}\.jpg$'  # Regular expression for "000001.jpg" format
    return re.match(pattern, filename) is not None

# Check for invalid filenames
invalid_filenames = df[~df['Filename'].apply(is_valid_filename)]

# Print rows with invalid filenames
for index, row in invalid_filenames.iterrows():
    print(f"Invalid filename '{row['Filename']}' in row {index + 1}")

# If you want to remove rows with invalid filenames from the DataFrame, you can use the following line
# df = df[df['Filename'].apply(is_valid_filename)]'''

In [ ]:
# Define list of class names (subset of the classes listed in `list_attr_celeba.txt`)
class_names = ["High_Cheekbones", "Mouth_Slightly_Open", "Smiling"]

# Reproducability
def set_seed(seed=31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed()

def custom_augmentation(np_tensor):

  def random_contrast(np_tensor):
    return tf.image.random_contrast(np_tensor, 0.5, 2)

  augmnted_tensor = random_contrast(np_tensor)
  return np.array(augmnted_tensor)

# Load training and validation sets
datagen=ImageDataGenerator(rescale=1./255.,
                          rotation_range=45,
                          horizontal_flip=True,
                          vertical_flip=True,
                          preprocessing_function=custom_augmentation)
valid_datagen=ImageDataGenerator(rescale=1./255.,
                                rotation_range=45,
                                horizontal_flip=True,
                                vertical_flip=True,
                                preprocessing_function=custom_augmentation)
test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator = datagen.flow_from_dataframe(
    dataframe=train,
    directory=img_dir,
    x_col='Filename',
    y_col=class_names,
    save_format='jpg',
    batch_size=30,
    seed=27,
    shuffle=True,
    class_mode="raw",
    target_size=(178, 218)
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid,
    directory=img_dir,
    x_col='Filename',
    y_col=class_names,
    save_format='jpg',
    batch_size=30,
    seed=9,
    shuffle=True,
    class_mode="raw",
    target_size=(178, 218)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=img_dir,
    x_col='Filename',
    batch_size=10,
    seed=3,
    shuffle=False,
    class_mode=None,
    target_size=(178, 218)
)

def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([30, 178, 218, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)

ds_valid = tf.data.Dataset.from_generator(
    lambda: valid_generator,
    output_types=(tf.float32, tf.float32),
    output_shapes=([30, 178, 218, 3], [None, len(class_names)])
).map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
import tensorflow.keras.backend as K
K.clear_session()

In [ ]:
from tensorflow.keras.models import Model

input_tensor = Input(shape=(218, 178, 3), name='input_01')
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_tensor)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Dropout(0.15)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.25)(x)

output1 = tf.keras.layers.Dense(1, activation='softmax', name='High_Cheekbones')(x)
output2 = tf.keras.layers.Dense(1, activation='softmax', name='Mouth_Slightly_Open')(x)
output3 = tf.keras.layers.Dense(1, activation='softmax', name='Smiling')(x)

model = tf.keras.models.Model(inputs=input_tensor, outputs=[output1, output2, output3,])

# Compile the model
model.compile(
    optimizer='adam',
    loss={'High_Cheekbones': 'binary_crossentropy', 'Mouth_Slightly_Open': 'binary_crossentropy', 'Smiling': 'binary_crossentropy'},
    metrics=['accuracy']
)

# Print model summary
model.summary()

In [ ]:
# save model summary to file
with open('/content/drive/My Drive/CelebA/Saved_models/VGG16_Transfer_3_Classes/metrics/summary.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

check_save_dir = '/content/drive/My Drive/CelebA/VGG16_Checkpoints/Checkpoints_Frozen_Uncropped/'
gc.enable()

checkpoint = ModelCheckpoint(
    check_save_dir,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss',
    mode='min',
    verbose=1
    )

# define early stopping callback
early_stopping = EarlyStopping(
    monitor='loss', # metric to monitor for early stopping
    patience=3, # number of epochs to wait before stopping
    restore_best_weights=True, # restore the best model weights found during training
)

In [ ]:
def generator_wrapper(dataset):
    for batch_x, batch_y in dataset:
        yield (batch_x, [batch_y[:, i] for i in range(3)])

ds_train_wrapped = generator_wrapper(ds_train)
ds_valid_wrapped = generator_wrapper(ds_valid)

In [ ]:
%cd drive/My Drive/CelebA

In [ ]:
model.load_weights('./Checkpoints_3_Classes/')

Train Data: 6,300 rows   
Train Batch Size: 30 images  
Train Step Size = $6300/30 = 210$  

-------
Valid Data: 2160 rows  
Valid Batch Size: 30 images  
Valid Step Size: $2160 / 30 = 72$  

Epoch, Epoch time, Step time  
1, 9870s, 47s   
2, 6017s, 28s  
3, 162s, 0.776s   
4, 160s, 0.765s  
5, 156s, 0.746s  
6, 149s, 0.711s  
7, 137s, 0.653s  
8, 131s, 0.624s  


In [ ]:
history = model.fit(
                    ds_train_wrapped,
                    epochs=10,
                    validation_data=ds_valid_wrapped,
                    batch_size=30,
                    steps_per_epoch=210,
                    validation_steps=72,
                    verbose=1,
                    callbacks=[checkpoint, early_stopping])

In [ ]:
history.history

In [ ]:
gc.collect()

In [ ]:
tf.keras.saving.save_model(
    model, './Saved_models/VGG16_Transfer_3_Classes/8_epochs_uncropped', overwrite=False, save_format='tf',
)

In [ ]:
reconstructed_model = keras.models.load_model("./Saved_models/Set_B/model_15")

In [ ]:
reconstructed_model.summary()

In [ ]:
reconstructed_model.load_weights('./Checkpoints/')

In [ ]:
history = reconstructed_model.fit(
                    ds_train_wrapped,
                    epochs=1,
                    validation_data=ds_valid_wrapped,
                    batch_size=25,
                    steps_per_epoch=910,
                    validation_steps=312,
                    verbose=1,
                    callbacks=[checkpoint, early_stopping])

In [ ]:
tf.keras.saving.save_model(
    reconstructed_model, './Saved_models/Set_B/model_15', overwrite=False, save_format='tf',
)

In [ ]:
test_generator.reset()
# Get the predicted probabilities for each class
predictions = model.predict(test_generator, steps=1)

In [ ]:
from PIL import Image
from textwrap import wrap
imagePreds = []

#print(predictions[20][14][0])
for i in range(3):
  classPreds = []
  for j in range(10):
    classPreds.append(predictions[i][j][0])
  imagePreds.append(classPreds)

class_names = ["High_Cheekbones", "Mouth_Slightly_Open", "Smiling"]
filenames = test['Filename'][:10]
#print(imagePreds)

for i in range(10):
  image = []
  for j in range(3):
    image.append(imagePreds[j][i])
  #print(image)
  sorted_indices = np.argsort(image)
  #print(sorted_indices)
  top_three_indices = sorted_indices[::-1][:3]
  #print(top_three_indices[0])
  top_three_arr = [image[x] for x in top_three_indices]
  #print(top_three_arr)
  xLabel = f"{class_names[sorted_indices[2]]}, {'%.2f' % (top_three_arr[2]*100)}%; {class_names[sorted_indices[1]]}, {'%.2f' % (top_three_arr[1]*100)}%; {class_names[sorted_indices[0]]}, {'%.2f' % (top_three_arr[0]*100)}%"
  #xLabel = ['\n'.join(wrap(l, 20)) for l in xLabel]
  # Plot the results
  plt.figure(figsize=(6, 12))
  #plt.subplot(16, 14, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(Image.open(os.path.join(img_dir, filenames.iloc[i])))
  plt.xlabel(xLabel)
  plt.tight_layout()
  plt.show()


In [ ]:
# set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=16, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore")

# convert history.history object to dataframe
history_frame = pd.DataFrame(history.history)


In [ ]:
print(history_frame)
history_frame.to_csv('/content/drive/My Drive/CelebA/Saved_models/VGG16_Transfer_3_Classes/metrics/8_epochs_uncropped.csv', index=False)

In [ ]:

history_frame.loc[:, [
                  'val_High_Cheekbones_accuracy',
                  'val_Smiling_accuracy',
                  'val_Mouth_Slightly_Open_accuracy',
                     ]].plot()
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend(loc='center left')
plt.show()

In [ ]:
val_accuracies = []
for col_name in history_frame.columns:
    if col_name.startswith('val_') and col_name.endswith('accuracy'):
        val_accuracies.append(history_frame[col_name])


In [ ]:
val_losses = []
for col_name in history_frame.columns:
    if col_name.startswith('val_') and col_name.endswith('loss'):
        val_losses.append(history_frame[col_name])

In [ ]:
import numpy as np
avg_val_losses = np.mean(val_losses, axis=0)
avg_val_accuracies = np.mean(val_accuracies, axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(avg_val_losses)
plt.title('Average Validation Loss. vs. Epochs (epochs 1-8)')
plt.xlabel('Epochs')
plt.ylabel('Average Validation Loss')
plt.show()

plt.plot(avg_val_accuracies)
plt.title('Average Validation Acc. vs. Epochs (epochs 1-8)')
plt.xlabel('Epochs')
plt.ylabel('Average Validation Accuracy')
plt.show()
